In [1]:
import os
import torch
import argparse
import numpy as np
import torch.utils.data
from easydict import EasyDict as edict
from timeit import default_timer as timer

from utils.eval import Metric
from utils.gpu_dispatch import GPU
from utils.common_utils import dir_check, to_device, ws, unfold_dict, dict_merge, GpuId2CudaId, Logger

from algorithm.dataset import CleanDataset, TrafficDataset
from algorithm.diffGSL.model import DiffSTG, save2file

In [2]:
def default_config(data='AIR_BJ'):
    config = edict()
    config.PATH_MOD = ws + '/output/model/'
    config.PATH_LOG = ws + '/output/log/'
    config.PATH_FORECAST = ws + '/output/forecast/'

    # Data Config
    config.data = edict()
    config.data.name = data
    config.data.path = ws + '/data/dataset/'
    config.graph_diffusion_step = 3


    config.data.feature_file = config.data.path + config.data.name + '/flow.npy'  # Add this line
    config.data.spatial = config.data.path + config.data.name + '/adj.npy'
    config.data.num_recent = 1

    # Data settings for different datasets
    if config.data.name == 'PEMS08':
        config.data.num_vertices = 170
        config.data.points_per_hour = 12
        config.data.val_start_idx = int(17856 * 0.6)
        config.data.test_start_idx = int(17856 * 0.8)

    if config.data.name == "AIR_BJ":
        config.data.num_vertices = 34
        config.data.points_per_hour = 1
        config.data.val_start_idx = int(8760 * 0.6)
        config.data.test_start_idx = int(8760 * 0.8)

    if config.data.name == 'AIR_GZ':
        config.data.num_vertices = 41
        config.data.points_per_hour = 1
        config.data.val_start_idx = int(8760 * 10 / 12)
        config.data.test_start_idx = int(8160 * 11 / 12)

    gpu_id = GPU().get_usefuel_gpu(max_memory=6000, condidate_gpu_id=[0])
    config.gpu_id = gpu_id
    if gpu_id != None:
        cuda_id = GpuId2CudaId(gpu_id)
        torch.cuda.set_device(f"cuda:{cuda_id}")
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = torch.device('cpu')
    # print(device)

    # Model config
    config.model = edict()
    config.model.T_p = 12
    config.model.T_h = 12
    config.model.V = config.data.num_vertices
    config.model.F = 1
    config.model.week_len = 7
    config.model.day_len = config.data.points_per_hour * 24
    config.model.device = device
    config.model.d_h = 32
    config.cheb_k = 3

    # Diffusion model config
    config.model.N = 200
    config.model.sample_steps = 10
    config.model.epsilon_theta = 'GSTNet'
    config.model.is_label_condition = True
    config.model.beta_end = 0.02
    config.model.beta_schedule = 'quad'
    config.model.sample_strategy = 'ddpm'

    config.n_samples = 2
    config.model.channel_multipliers = [1, 2]
    config.model.supports_len = 2

    # Training config
    config.model_name = 'DiffSTG'
    config.is_test = False
    config.epoch = 300
    config.optimizer = "adam"
    config.lr = 1e-4
    config.batch_size = 32
    config.wd = 1e-5
    config.early_stop = 10
    config.start_epoch = 0
    config.device = device
    config.logger = Logger()

    if not os.path.exists(config.PATH_MOD):
        os.makedirs(config.PATH_MOD)
    if not os.path.exists(config.PATH_LOG):
        os.makedirs(config.PATH_LOG)
    if not os.path.exists(config.PATH_FORECAST):
        os.makedirs(config.PATH_FORECAST)
    return config

In [3]:
config = default_config("AIR_BJ")

clean_data = CleanDataset(config)
train_dataset = TrafficDataset(clean_data, (0 + config.model.T_p, config.data.val_start_idx - config.model.T_p + 1), config)
train_loader = torch.utils.data.DataLoader(train_dataset, config.batch_size, shuffle=True, pin_memory=True)

nvidia-smi > /Users/jcy/Desktop/DiffSTG-main/output/gpustat//gpustat.txt
wrong in load gpu info dict list index out of range
None gpu is avalible, try again later
sample num: 5233


sh: nvidia-smi: command not found


In [4]:
def prepare_x_y(x, y):
    """
    :param x: shape (batch_size, seq_len, num_sensor, input_dim)
    :param y: shape (batch_size, horizon, num_sensor, input_dim)
    :return1: x shape (seq_len, batch_size, num_sensor, input_dim)
              y shape (horizon, batch_size, num_sensor, input_dim)
    :return2: x: shape (seq_len, batch_size, num_sensor * input_dim)
              y: shape (horizon, batch_size, num_sensor * output_dim)
    """
    x0 = x[..., :1]
    y0 = y[..., :1]
    y1 = y[..., 1:]
    return x0.to(config.device), y0.to(config.device), y1.to(config.device) # x, y, y_cov

In [5]:
for i, batch in enumerate(train_loader):
    b = batch
future, history, pos_w, pos_d = b

In [6]:
x0 = torch.cat((history, future), dim=1).to(config.device) #  (B, T, V, F)

In [7]:
config.mask_ratio = 0.0
config.N = 10
config.sample_steps = 10
config.sample_strategy = 'ddpm'
config.beta_schedule = 'quad'
config.num_layers = 1
config.V = 34
config.T_h = 12

In [8]:
mask =  torch.randint_like(history, low=0, high=100) < int(config.mask_ratio * 100) # mask the history in a ratio with mask_ratio
history[mask] = 0
x_masked = torch.cat((history, torch.zeros_like(future)), dim=1).to(config.device) # (B, T, V, F)

# debugging of graphnet

In [9]:
from algorithm.diffgraph.graphnet_ver2_copy import GraphNet

In [10]:
eps_model = GraphNet(config, rnn_num_units= 16, input_dim= 1, y_cov_dim=1, out_dim=1)

In [11]:
def gather(consts: torch.Tensor, t: torch.Tensor):
    c = consts.gather(-1, t)
    return c.reshape(-1, 1, 1, 1)

In [12]:
def q_xt_x0( x0: torch.Tensor, t: torch.Tensor, eps): # forward diffusion process
    """
    Sample from  q(x_t|x_0) ~ N(x_t; \sqrt\bar\alpha_t * x_0, (1 - \bar\alpha_t)I) 
    """
    if eps is None:
        eps = torch.randn_like(x0)

    mean = gather(alpha_bar, t) ** 0.5 * x0
    var = 1 - gather(alpha_bar, t)

    return mean + eps * (var ** 0.5)

In [13]:
beta = torch.linspace(0.001 ** 0.5, 0.02 ** 0.5, 10) ** 2
beta = beta.to(config.device)
alpha = 1.0 - beta
alpha_bar = torch.cumprod(alpha, dim=0)

In [14]:
t = torch.randint(0, 10, (x0.shape[0],), device=x0.device, dtype=torch.long)
eps = torch.randn_like(x0)
xt = q_xt_x0(x0, t, eps)

In [15]:
print(t.size()) # time 
print(eps.size()) # epsilon target
print(xt.size()) # noise

torch.Size([17])
torch.Size([17, 24, 34, 1])
torch.Size([17, 24, 34, 1])


In [16]:
x_masked.size()

torch.Size([17, 24, 34, 1])

In [17]:
x = torch.cat((history, xt[:,:config.T_h,:,:]), dim=3)

In [18]:
x.size()

torch.Size([17, 12, 34, 2])

In [19]:
import torch.nn as nn
x_proj = nn.Conv2d(2, 1, (1,1))

In [20]:
x_P = x_proj(x.transpose(1,3))
x_P = x_P.transpose(1,3)

In [21]:
x_P.size()

torch.Size([17, 12, 34, 1])

In [22]:
eps_theta = eps_model(x_P, xt)

torch.Size([17, 12, 34, 16])


In [23]:
eps_theta.size()

torch.Size([17, 24, 34, 1])

In [24]:
torch.min(eps_theta)

tensor(-2.5992, grad_fn=<MinBackward1>)

In [26]:
from torch.nn import functional as F
F.mse_loss(eps, eps_theta)

tensor(1.3794, grad_fn=<MseLossBackward0>)

In [27]:
total_params = sum(p.numel() for p in eps_model.parameters())
print(f"총 파라미터 수 : {total_params}")

총 파라미터 수 : 60898


In [ ]:
import torch.nn as nn
x_proj = nn.Conv2d(2, 1, (1,1))

In [ ]:
x_proj = nn.Conv2d(2, 1, (1,1))

In [ ]:
_masked, pos_w, pos_d = c  # x_masked: (B, F, V, T), pos_w: (B,T,1,1), pos_d: (B,T,1,1)

x = torch.cat((x, x_masked), dim=3) # (B, F, V, 2 * T) noise + history

x = self.x_proj(x)

t = TimeEmbedding(t, self.d_h)

h = [x]

NameError: name 'c' is not defined

In [ ]:
def loss(self, x0: torch.Tensor, c: Tuple):
        """
        Loss calculation
        x0: (B, ...) : history + futrue
        c: The condition, c is a tuple of torch tensor, here c = (feature, pos_w, pos_d)
        """
        #
        t = torch.randint(0, self.N, (x0.shape[0],), device=x0.device, dtype=torch.long)

        # Note that in the paper, t \in [1, T], but in the code, t \in [0, T-1]
        eps = torch.randn_like(x0)

        xt = self.q_xt_x0(x0, t, eps)
        eps_theta = self.eps_model(xt, t, c)
        return F.mse_loss(eps, eps_theta)

In [ ]:
def loss(self, x0: torch.Tensor, c: Tuple):
        """
        Loss calculation
        x0: (B, ...) : history + futrue
        c: The condition, c is a tuple of torch tensor, here c = (feature, pos_w, pos_d)
        """
        #
        t = torch.randint(0, self.N, (x0.shape[0],), device=x0.device, dtype=torch.long)

        # Note that in the paper, t \in [1, T], but in the code, t \in [0, T-1]
        eps = torch.randn_like(x0)

        xt = self.q_xt_x0(x0, t, eps) # noisng of history + future

        # x_masked, y, y_cov = c
        xt = xt.transpose(1,3)
        xt_masked = xt[:,:self.config.T_h,:,:]
        # x_masked = x_masked.transpose(1,3)
        # xt = xt.transpose(1,3)

        eps_theta = self.eps_model(xt_masked, xt)
        loss = nn.MSELoss()

        # return F.mse_loss(eps, eps_theta)
        return loss(eps.transpose(1,3)[:,-self.config.T_h:,:,], eps_theta)